In [ ]:

from classes.event_graph import EventGraph
from classes.event_node import EventNode
from data.event_types import event_types

n0 = EventNode(event_types[0]) 
n1 = EventNode(event_types[1]) 
n2 = EventNode(event_types[2])

print("----------------------------")

eg = EventGraph()
eg.add_node(n0)
eg.add_node(n1)
eg.add_node(n2)
eg.add_edge(n0, n1)
eg.add_edge(n1, n2)
eg.visualize()

In [125]:
import hashlib

def stable_hash(label: str):
    h = hashlib.blake2b(label.encode("utf-8"), digest_size=16).digest()
    return int.from_bytes(h, "little")

def WL_neighborhood_labels(graph: EventGraph, iterations: int = 3, inspect: bool = False):
    labels = {node: stable_hash(graph.nodes[node].event_type) for node in graph.nodes}
    history = [labels.copy()]
    
    for _ in range(iterations):
        new_labels = {}
        for node in graph.nodes:
            nbrs = graph.adj_list.get(node, [])  # safe default
            neighbor_labels = [labels[nbr] for nbr in nbrs if nbr in labels]
            
            # deterministic sort even if labels later become non-strings
            neighbor_labels.sort(key=repr)

            signature = (labels[node], tuple(neighbor_labels))

            if inspect:
                # keep inspection output readable and still a string so next iter is safe
                new_label = signature
            else:
                new_label = stable_hash(repr(signature))
            
            new_labels[node] = new_label
            
        labels = new_labels
        history.append(labels.copy())
        
    return history  # list of dicts for it=0..h

In [126]:
labels = WL_neighborhood_labels(eg)
print(labels)

[{'eeb9c9258857432b8e5499d367674b53': 93433054168679946075865275642629393240, '550c64a6e0b44acc9929d2ca49fc777e': 65701059784870561186292905343980779203, 'b47371f81e4b4e26a70b09a562a221eb': 158219164025945609616885530571600077192}, {'eeb9c9258857432b8e5499d367674b53': 210689940475976901938048864362768591456, '550c64a6e0b44acc9929d2ca49fc777e': 195050179273803116973085247012978721960, 'b47371f81e4b4e26a70b09a562a221eb': 156948249707131972314697569980015481027}, {'eeb9c9258857432b8e5499d367674b53': 216991367043714769640016947020418008246, '550c64a6e0b44acc9929d2ca49fc777e': 200870902363397443375183181182280170270, 'b47371f81e4b4e26a70b09a562a221eb': 29641278237178254485597453965643685284}, {'eeb9c9258857432b8e5499d367674b53': 252166089463993543676619844122931078035, '550c64a6e0b44acc9929d2ca49fc777e': 105063728035190742098303524639375851287, 'b47371f81e4b4e26a70b09a562a221eb': 320455612623921187340720680842695643592}]


In [ ]:
def graph_to_fingerprint(graph, D=1024):
    all_labels = WL_neighborhood_labels(graph)
    fingerprint = [0] * D
    for labels in all_labels:          # use 0..h
        for feat in labels.values():
            idx = feat % D
            fingerprint[idx] = 1       # or +=1 for counts
    return fingerprint

In [143]:
fingerprint = graph_to_fingerprint(eg)
print(fingerprint)

856
707
392
608
168
195
182
798
420
915
791
456
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 